In [2]:
import os
from collections import Counter, defaultdict
from PIL import Image
import numpy as np
import random
import shutil
from pathlib import Path

## Rohdaten verteilen

# Es wird davon ausgegangen, dass der von kaggle heruntergeladene .zip Ordner (Vorgehen beschrieben in der readme.md) im Ordner projekt_emotions entpackt wurde und nun ein Ordner "facial_emotion_dataset 2" vorhanden ist!!

In [4]:
# Pfad relativ zum Notebook im scripts-Ordner
folder_to_delete = Path("../facial_emotion_dataset 2/dataset/Ahegao")

# Überprüfen, ob der Ordner existiert und löschen
if folder_to_delete.exists() and folder_to_delete.is_dir():
    shutil.rmtree(folder_to_delete)
    print(f"Ordner {folder_to_delete} wurde gelöscht.")
else:
    print(f"Ordner {folder_to_delete} existiert nicht.")


Ordner ../facial_emotion_dataset 2/dataset/Ahegao wurde gelöscht.


In [5]:
# Quell- und Zielpfade definieren
source_root = Path("../facial_emotion_dataset 2")
source_dataset = source_root / "dataset"
target_dir = Path("../emotion_dataset")

# Zielordner erstellen, falls nicht vorhanden
target_dir.mkdir(parents=True, exist_ok=True)

# Alle Unterordner im Quell-Dataset verschieben
for subfolder in source_dataset.iterdir():
    if subfolder.is_dir():
        target_subfolder = target_dir / subfolder.name
        print(f"Verschiebe {subfolder.name} → {target_subfolder}")
        shutil.move(str(subfolder), str(target_subfolder))

# Überprüfen, ob der Ursprungsordner nun leer ist und löschen
try:
    shutil.rmtree(source_root)
    print(f"Ordner {source_root} wurde vollständig gelöscht.")
except Exception as e:
    print(f"Fehler beim Löschen von {source_root}: {e}")


Verschiebe Happy → ../emotion_dataset/Happy
Verschiebe Sad → ../emotion_dataset/Sad
Verschiebe Surprise → ../emotion_dataset/Surprise
Verschiebe Neutral → ../emotion_dataset/Neutral
Verschiebe Angry → ../emotion_dataset/Angry
Ordner ../facial_emotion_dataset 2 wurde vollständig gelöscht.


In [6]:
# Quell- und Zielverzeichnisse definieren
source_dir = Path("../emotion_dataset")
target_dir = Path("../emotion_dataset_v2")

# Zielordner erstellen, falls er noch nicht existiert
target_dir.mkdir(parents=True, exist_ok=True)

# Liste der zu kopierenden Ordner
folders_to_copy = ["Happy", "Sad"]

# Kopieren der Ordner
for folder_name in folders_to_copy:
    source_path = source_dir / folder_name
    target_path = target_dir / folder_name

    if source_path.exists() and source_path.is_dir():
        shutil.copytree(source_path, target_path, dirs_exist_ok=True)
        print(f"{folder_name} → {target_path} wurde kopiert.")
    else:
        print(f"{folder_name} existiert nicht im Quellordner.")

print("Kopiervorgang abgeschlossen.")

Happy → ../emotion_dataset_v2/Happy wurde kopiert.
Sad → ../emotion_dataset_v2/Sad wurde kopiert.
Kopiervorgang abgeschlossen.


## emotion_dataset vorbereiten

In [ ]:
# relativer Pfad ausgehend vom Notebook in /scripts
DATA_DIR = Path("../emotion_dataset")

In [3]:
#Untersuchung des Datensatzes: Pixel, Format, Farbe,...

# Listen für Bildgrößen, Formate und Modi
widths = []
heights = []
formats = []
modes = []

# Dictionary für Bildanzahl je Unterordner
folder_counts = defaultdict(int)

# Bilder durchgehen
for root, _, files in os.walk(DATA_DIR):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img_path = os.path.join(root, file)
            try:
                with Image.open(img_path) as img:
                    widths.append(img.width)
                    heights.append(img.height)
                    formats.append(img.format)
                    modes.append(img.mode)
                    # Ordnername extrahieren (relativ zum data_dir)
                    relative_folder = os.path.relpath(root, DATA_DIR)
                    folder_counts[relative_folder] += 1
            except Exception as e:
                print(f"Fehler beim Laden von {img_path}: {e}")

# Ergebnisse ausgeben
if widths and heights:
    print(f"Anzahl gesamt: {len(widths)}")
    print(f"Breite - min: {np.min(widths)}, max: {np.max(widths)}, Ø: {np.mean(widths):.2f}")
    print(f"Höhe  - min: {np.min(heights)}, max: {np.max(heights)}, Ø: {np.mean(heights):.2f}")
    
    format_counts = Counter(formats)
    print("\nFormate:")
    for fmt, count in format_counts.items():
        print(f"{fmt}: {count}")
    
    mode_counts = Counter(modes)
    print("\nFarbe (Bildmodi):")
    for mode, count in mode_counts.items():
        print(f"{mode}: {count}")

    print("\nAnzahl je Kategorie:")
    for folder, count in folder_counts.items():
        print(f"{folder}: {count}")
else:
    print("Keine Bilder gefunden oder Fehler beim Laden.")


# Intervalle für die Größenkategorien
def count_in_intervals(values, intervals):
    counter = Counter()
    for val in values:
        for limit in intervals:
            if val <= limit:
                counter[limit] += 1
                break
    return counter

intervals = [200, 400, 600, 1000, 1500, 2000, 3000, 4000]
width_counts = count_in_intervals(widths, intervals)
height_counts = count_in_intervals(heights, intervals)

# Ausgabe der Ergebnisse
if widths and heights:
    print(f"Anzahl der analysierten Bilder: {len(widths)}")
    print(f"Minimale Breite: {np.min(widths)}, Maximale Breite: {np.max(widths)}, Durchschnittliche Breite: {np.mean(widths):.2f}")
    print(f"Minimale Höhe: {np.min(heights)}, Maximale Höhe: {np.max(heights)}, Durchschnittliche Höhe: {np.mean(heights):.2f}")
    
    print("\nAnzahl der Bilder je Breitenbereich:")
    for limit in intervals:
        print(f"Bis {limit} Pixel: {width_counts[limit]}")
    
    print("\nAnzahl der Bilder je Höhenbereich:")
    for limit in intervals:
        print(f"Bis {limit} Pixel: {height_counts[limit]}")
else:
    print("Keine Bilder gefunden oder Fehler beim Laden.")




Anzahl gesamt: 14248
Breite - min: 8, max: 4414, Ø: 412.42
Höhe  - min: 12, max: 4414, Ø: 450.82

Formate:
JPEG: 8445
PNG: 5798
BMP: 5

Farbe (Bildmodi):
RGB: 14248

Anzahl je Kategorie:
Happy: 3740
Sad: 3934
Surprise: 1234
Neutral: 4027
Angry: 1313
Anzahl der analysierten Bilder: 14248
Minimale Breite: 8, Maximale Breite: 4414, Durchschnittliche Breite: 412.42
Minimale Höhe: 12, Maximale Höhe: 4414, Durchschnittliche Höhe: 450.82

Anzahl der Bilder je Breitenbereich:
Bis 200 Pixel: 3469
Bis 400 Pixel: 5961
Bis 600 Pixel: 2217
Bis 1000 Pixel: 1755
Bis 1500 Pixel: 531
Bis 2000 Pixel: 203
Bis 3000 Pixel: 101
Bis 4000 Pixel: 10

Anzahl der Bilder je Höhenbereich:
Bis 200 Pixel: 2272
Bis 400 Pixel: 6303
Bis 600 Pixel: 2679
Bis 1000 Pixel: 2072
Bis 1500 Pixel: 585
Bis 2000 Pixel: 217
Bis 3000 Pixel: 106
Bis 4000 Pixel: 13


In [4]:
# Konfiguration
min_width = 200
max_width = 600
min_height = 200
max_height = 600
max_keep = 500

# Statistik-Zähler
deleted_due_to_size = defaultdict(int)
deleted_due_to_limit = defaultdict(int)


# Teil 1: Bilder löschen, die bmp format haben oder zu klein oder zu groß sind
for root, _, files in os.walk(DATA_DIR):
    for file in files:
        if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp')):
            img_path = os.path.join(root, file)
            #bmp bilder löschen
            if file.lower().endswith(".bmp"):
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Gelöscht: {file_path}")
                except Exception as e:
                    print(f"Fehler beim Löschen von {file_path}: {e}")
            try: #größen löschen
                with Image.open(img_path) as img:
                    width, height = img.size
                    total_pixels = width * height
                    if width < min_width or width > max_width or height < min_height or height > max_height:
                        os.remove(img_path)
                        folder_name = os.path.basename(root)
                        deleted_due_to_size[folder_name] += 1
            except Exception as e:
                print(f"Fehler beim Verarbeiten von {img_path}: {e}")

# Teil 2: Zufällig Bilder löschen, um auf max_keep pro Ordner zu begrenzen
for folder in os.listdir(DATA_DIR):
    folder_path = os.path.join(DATA_DIR, folder)
    if os.path.isdir(folder_path):
        images = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        if len(images) > max_keep:
            to_keep = set(random.sample(images, max_keep))
            for img in images:
                if img not in to_keep:
                    os.remove(os.path.join(folder_path, img))
                    deleted_due_to_limit[folder] += 1

# Ausgabe der Statistik
print("\nLösch-Statistik:")
for folder in sorted(os.listdir(DATA_DIR)):
    if os.path.isdir(os.path.join(DATA_DIR, folder)):
        size_del = deleted_due_to_size.get(folder, 0)
        limit_del = deleted_due_to_limit.get(folder, 0)
        print(f"- {folder}: {size_del} wegen Größe gelöscht, {limit_del} zusätzlich begrenzt auf 500 Bilder.")



Gelöscht: /Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset/Happy/2b36e0f43db9afb8112bbebd145789a4f6c35a7098eec295f0237d20.bmp
Fehler beim Verarbeiten von /Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset/Happy/2b36e0f43db9afb8112bbebd145789a4f6c35a7098eec295f0237d20.bmp: [Errno 2] No such file or directory: '/Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset/Happy/2b36e0f43db9afb8112bbebd145789a4f6c35a7098eec295f0237d20.bmp'
Gelöscht: /Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset/Sad/0b1040152218828705966fe7b6ae4d84ef194c3fec5a8853b64aa14f.bmp
Fehler beim Verarbeiten von /Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset/Sad/0b1040152218828705966fe7b6ae4d84ef194c3fec5a8853b64aa14f.bmp: [Errno 2] No such file or directory: '/Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset/Sad/0b1040152218828705966fe7b6ae4d84ef194c3fec5a8853b64aa14f.bmp'
Gelöscht: /Users

In [6]:
# Bildformate, die gezählt werden sollen
image_extensions = (".jpg", ".jpeg", ".png")

# Zähler
folder_counts = defaultdict(int)
total_images = 0

# Durch Unterordner gehen und zählen
for root, _, files in os.walk(DATA_DIR):
    count = sum(1 for file in files if file.lower().endswith(image_extensions))
    if count > 0:
        relative_folder = os.path.relpath(root, DATA_DIR)
        folder_counts[relative_folder] += count
        total_images += count

# Ausgabe
print("Bilderanzahl je Unterordner:")
for folder, count in folder_counts.items():
    print(f"{folder}: {count}")

print(f"\nGesamtanzahl der Bilder: {total_images}")


Bilderanzahl je Unterordner:
Happy: 500
Sad: 500
Surprise: 500
Neutral: 500
Angry: 500

Gesamtanzahl der Bilder: 2500


In [7]:
def train_test_split(DATA_DIR): #data_dir s.o.
    print("--- Train-Test-Split gestartet ---")

    # Alle Klassenverzeichnisse ermitteln (Unterordner in 'data_dir' sind die Klassen)
    classes_dir = [name for name in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, name))]
    
    # Zielverzeichnis für den Split (erstellt "train" und "test" in "garbage-datasets")
    root_dir = os.path.join(DATA_DIR, "datasets")
    train_root = os.path.join(root_dir, "train")
    test_root = os.path.join(root_dir, "test")

    test_ratio = 0.2
    list_train, list_test = [], []

    for cls in classes_dir:
        print(f"--- Klassenname: {cls} ---")
        src = os.path.join(DATA_DIR, cls)

        # Alle Bilddateien auflisten
        allFileNames = [f for f in os.listdir(src) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        np.random.shuffle(allFileNames)

        # Aufteilen in Training und Test
        split_idx = int(len(allFileNames) * (1 - test_ratio))
        train_FileNames, test_FileNames = allFileNames[:split_idx], allFileNames[split_idx:]

        print(f'Bilder insgesamt: {len(allFileNames)}')
        print(f'Training: {len(train_FileNames)}')
        print(f'Testing: {len(test_FileNames)}')

        list_train.append(len(train_FileNames))
        list_test.append(len(test_FileNames))

        # Zielverzeichnisse für die Klasse erstellen
        train_dir = os.path.join(train_root, cls)
        test_dir = os.path.join(test_root, cls)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        # Dateien mit vollständigem Pfad kopieren
        for name in train_FileNames:
            src_path = os.path.join(src, name)
            dest_path = os.path.join(train_dir, name)
            if os.path.exists(src_path):
                shutil.copy(src_path, dest_path)

        for name in test_FileNames:
            src_path = os.path.join(src, name)
            dest_path = os.path.join(test_dir, name)
            if os.path.exists(src_path):
                shutil.copy(src_path, dest_path)

    print("--- Train-Test-Split fertig ---")
    print(f"--- In train liegen {sum(list_train)} Bilder insgesamt ---")
    print(f"--- In test liegen {sum(list_test)} Bilder insgesamt ---")
    return root_dir

# Train-Test-Split ausführen
dataset_path = train_test_split(DATA_DIR)


--- Train-Test-Split gestartet ---
--- Klassenname: Happy ---
Bilder insgesamt: 500
Training: 400
Testing: 100
--- Klassenname: Sad ---
Bilder insgesamt: 500
Training: 400
Testing: 100
--- Klassenname: Surprise ---
Bilder insgesamt: 500
Training: 400
Testing: 100
--- Klassenname: Neutral ---
Bilder insgesamt: 500
Training: 400
Testing: 100
--- Klassenname: Angry ---
Bilder insgesamt: 500
Training: 400
Testing: 100
--- Train-Test-Split fertig ---
--- In train liegen 2000 Bilder insgesamt ---
--- In test liegen 500 Bilder insgesamt ---


## kleineren Datensatz emotion_dataset_v2 vorbereiten

In [8]:
# relativer Pfad ausgehend vom Notebook in /scripts
DATA_DIR_v2 = Path("../emotion_dataset_v2")

In [9]:
from PIL import Image

# Liste für Breiten und Höhen der Bilder
widths = []
heights = []

# Durch alle Unterordner iterieren
for root, _, files in os.walk(DATA_DIR_v2):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  
            img_path = os.path.join(root, file)
            try:
                with Image.open(img_path) as img:
                    width, height = img.size
                    widths.append(width)
                    heights.append(height)
            except Exception as e:
                print(f"Fehler beim Laden von {img_path}: {e}")

# Berechnungen
if widths and heights:
    print(f"Anzahl der analysierten Bilder: {len(widths)}")
    print(f"Minimale Breite: {np.min(widths)}, Maximale Breite: {np.max(widths)}, Durchschnittliche Breite: {np.mean(widths):.2f}")
    print(f"Minimale Höhe: {np.min(heights)}, Maximale Höhe: {np.max(heights)}, Durchschnittliche Höhe: {np.mean(heights):.2f}")
else:
    print("Keine Bilder gefunden oder Fehler beim Laden.")


Anzahl der analysierten Bilder: 7674
Minimale Breite: 8, Maximale Breite: 4414, Durchschnittliche Breite: 438.64
Minimale Höhe: 12, Maximale Höhe: 4414, Durchschnittliche Höhe: 464.17


In [10]:
# Konfiguration
min_width = 200
max_width = 600
min_height = 200
max_height = 600
max_keep = 2000

# Statistik-Zähler
deleted_due_to_size = defaultdict(int)
deleted_due_to_limit = defaultdict(int)


# Teil 1: Bilder löschen, die bmp format haben oder zu klein oder zu groß sind
for root, _, files in os.walk(DATA_DIR_v2):
    for file in files:
        if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp')):
            img_path = os.path.join(root, file)
            #bmp bilder löschen
            if file.lower().endswith(".bmp"):
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Gelöscht: {file_path}")
                except Exception as e:
                    print(f"Fehler beim Löschen von {file_path}: {e}")
            try: #größen löschen
                with Image.open(img_path) as img:
                    width, height = img.size
                    total_pixels = width * height
                    if width < min_width or width > max_width or height < min_height or height > max_height:
                        os.remove(img_path)
                        folder_name = os.path.basename(root)
                        deleted_due_to_size[folder_name] += 1
            except Exception as e:
                print(f"Fehler beim Verarbeiten von {img_path}: {e}")

# Teil 2: Zufällig Bilder löschen, um auf max_keep pro Ordner zu begrenzen
for folder in os.listdir(DATA_DIR_v2):
    folder_path = os.path.join(DATA_DIR_v2, folder)
    if os.path.isdir(folder_path):
        images = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        if len(images) > max_keep:
            to_keep = set(random.sample(images, max_keep))
            for img in images:
                if img not in to_keep:
                    os.remove(os.path.join(folder_path, img))
                    deleted_due_to_limit[folder] += 1

# Ausgabe der Statistik
print("\nLösch-Statistik:")
for folder in sorted(os.listdir(DATA_DIR_v2)):
    if os.path.isdir(os.path.join(DATA_DIR_v2, folder)):
        size_del = deleted_due_to_size.get(folder, 0)
        limit_del = deleted_due_to_limit.get(folder, 0)
        print(f"- {folder}: {size_del} wegen Größe gelöscht, {limit_del} zusätzlich begrenzt auf 2000 Bilder.")



Gelöscht: /Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset_v2/Happy/2b36e0f43db9afb8112bbebd145789a4f6c35a7098eec295f0237d20.bmp
Fehler beim Verarbeiten von /Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset_v2/Happy/2b36e0f43db9afb8112bbebd145789a4f6c35a7098eec295f0237d20.bmp: [Errno 2] No such file or directory: '/Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset_v2/Happy/2b36e0f43db9afb8112bbebd145789a4f6c35a7098eec295f0237d20.bmp'
Gelöscht: /Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset_v2/Sad Kopie/0b1040152218828705966fe7b6ae4d84ef194c3fec5a8853b64aa14f.bmp
Fehler beim Verarbeiten von /Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset_v2/Sad Kopie/0b1040152218828705966fe7b6ae4d84ef194c3fec5a8853b64aa14f.bmp: [Errno 2] No such file or directory: '/Users/Mareikes/Desktop/Wahlpflicht_AI/projekt_emotions/emotion_dataset_v2/Sad Kopie/0b1040152218828705966fe7b6ae4d84ef194c3fec

In [11]:
def train_test_split(DATA_DIR_v2): #data_dir s.o.
    print("--- Train-Test-Split gestartet ---")

    # Alle Klassenverzeichnisse ermitteln (Unterordner in 'data_dir' sind die Klassen)
    classes_dir = [name for name in os.listdir(DATA_DIR_v2) if os.path.isdir(os.path.join(DATA_DIR_v2, name))]
    
    # Zielverzeichnis für den Split (erstellt "train" und "test" in "garbage-datasets")
    root_dir = os.path.join(DATA_DIR_v2, "datasets")
    train_root = os.path.join(root_dir, "train")
    test_root = os.path.join(root_dir, "test")

    test_ratio = 0.2
    list_train, list_test = [], []

    for cls in classes_dir:
        print(f"--- Klassenname: {cls} ---")
        src = os.path.join(DATA_DIR_v2, cls)

        # Alle Bilddateien auflisten
        allFileNames = [f for f in os.listdir(src) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        np.random.shuffle(allFileNames)

        # Aufteilen in Training und Test
        split_idx = int(len(allFileNames) * (1 - test_ratio))
        train_FileNames, test_FileNames = allFileNames[:split_idx], allFileNames[split_idx:]

        print(f'Bilder insgesamt: {len(allFileNames)}')
        print(f'Training: {len(train_FileNames)}')
        print(f'Testing: {len(test_FileNames)}')

        list_train.append(len(train_FileNames))
        list_test.append(len(test_FileNames))

        # Zielverzeichnisse für die Klasse erstellen
        train_dir = os.path.join(train_root, cls)
        test_dir = os.path.join(test_root, cls)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        # Dateien mit vollständigem Pfad kopieren
        for name in train_FileNames:
            src_path = os.path.join(src, name)
            dest_path = os.path.join(train_dir, name)
            if os.path.exists(src_path):
                shutil.copy(src_path, dest_path)

        for name in test_FileNames:
            src_path = os.path.join(src, name)
            dest_path = os.path.join(test_dir, name)
            if os.path.exists(src_path):
                shutil.copy(src_path, dest_path)

    print("--- Train-Test-Split fertig ---")
    print(f"--- In train liegen {sum(list_train)} Bilder insgesamt ---")
    print(f"--- In test liegen {sum(list_test)} Bilder insgesamt ---")
    return root_dir

# Train-Test-Split ausführen
dataset_path = train_test_split(DATA_DIR_v2)


--- Train-Test-Split gestartet ---
--- Klassenname: Happy ---
Bilder insgesamt: 1937
Training: 1549
Testing: 388
--- Klassenname: Sad Kopie ---
Bilder insgesamt: 2000
Training: 1600
Testing: 400
--- Train-Test-Split fertig ---
--- In train liegen 3149 Bilder insgesamt ---
--- In test liegen 788 Bilder insgesamt ---
